In [5]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [6]:
# Your client credentials
client_id = 'sh-099d49d0-fe42-48c7-aca8-62c326041498'
client_secret = 'XwrtUDIXG02um4aZM3DknqumRjxhRtdU'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)

b'[]'


In [7]:
resp

<Response [200]>